<a href="https://colab.research.google.com/github/FNSY96/TerrorismEventsPrediction/blob/master/Graduation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python Imports**
PLEASE PUT ALL LIBRARY IMPORTS IN THIS CELL **ONLY**

In [1]:
import pandas as pd
import csv
import glob
import os
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import io
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
print("imports run")

imports run


In [2]:
!pip3 install pandas

# Data Set Imports

In [3]:
!pip install -U -q PyDrive
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

    100% |████████████████████████████████| 993kB 17.3MB/s 


In [4]:
# drive = GoogleDrive(gauth)
drive.mount("/content/drive", force_remount=True)
datadir  = 'drive/My Drive/GraduationProject/Data'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


###**GTD**

In [0]:
# !ls drive/'My Drive'/GraduationProject/Data/gtd.csv

# Pandas Data Frames
Useful link : https://youtu.be/2AFGPdNn4FM

## **GTD Data Frame**

In [31]:
gtd_df = pd.read_csv(os.path.join(datadir, 'gtd.csv'),encoding='latin-1')
# print(gtd_df.columns.values)
# print(gtd_df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
gtd_filtered_df = gtd_df[['country_txt','region_txt','city','provstate', 'target1', 'iyear','imonth', 'gname', 'weaptype1_txt','attacktype1_txt','attacktype1']]
# print(gtd_filtered_df)
# print(gtd_filtered_df.isna().sum())

## **GECON Data Frame**

In [11]:
gecon_df = pd.read_csv(os.path.join(datadir, 'Gecon40_post_final_kh.csv'),encoding='latin-1')
# print(gecon_df.columns.values)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,19,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
gecon_filtered_df = gecon_df[['AREA', 'COUNTRY', 'DIS_LAKE','DIS_MAJOR_RIVER']]

# print(gecon_filtered_df.isna().sum())

# print(gecon_filtered_df)

## **GeoEPR Data Frame**

In [0]:
geo_df = pd.read_csv(os.path.join(datadir, 'GeoEPR-2018.1.csv'),encoding='latin-1')
# print(geo_df.columns.values)
# # statename, group
# print(geo_df)

In [0]:
geo_filtered_df = geo_df[['statename','group']]
geo_filtered_df = geo_filtered_df.rename(index=str, columns={"statename": "country_txt"})
# print(geo_filtered_df)
# print(geo_filtered_df.isna().sum())
# print(geo_filtered_df.groupby('country_txt').count())

In [0]:
# clean NaN
# geo_filtered_df = geo_filtered_df.fillna(geo_filtered_df.mean())
#attributes_NaN = ['statename', 'group', 'type']
#geo_filtered_df = remove_NaN(geo_filtered_df, attributes_NaN)

#print(geo_filtered_df)

## **Happiness World Report Data Frame**

In [0]:
happy_df_17 = pd.read_csv(os.path.join(datadir, 'Happiness2017.csv'),encoding='latin-1')
happy_df_16 = pd.read_csv(os.path.join(datadir, 'Happiness2016.csv'),encoding='latin-1')
happy_df_15 = pd.read_csv(os.path.join(datadir, 'Happiness2015.csv'),encoding='latin-1')
# happiness = [happy_df_15, happy_df_16, happy_df_17]
happy_df_17 = happy_df_17.rename(index=str, columns={"Country": "country_txt", "Happiness.Rank": "Happiness.Rank17", "Happiness.Score": "Happiness.Score17", "Freedom": "Freedom17", "Economy..GDP.per.Capita.": "Economy17"})
happy_df_16 = happy_df_16.rename(index=str, columns={"Country": "country_txt", "Happiness Rank": "Happiness.Rank16", "Happiness Score": "Happiness.Score16", "Freedom": "Freedom16", "Economy (GDP per Capita)": "Economy16"})
happy_df_15 = happy_df_15.rename(index=str, columns={"Country": "country_txt", "Happiness Rank": "Happiness.Rank15", "Happiness Score": "Happiness.Score15", "Freedom": "Freedom15", "Economy (GDP per Capita)": "Economy15"})

In [0]:
happiness_filtered = [happy_df_15[['country_txt','Happiness.Score15', 'Freedom15', 'Economy15']],
                      happy_df_16[['country_txt','Happiness.Score16', 'Freedom16', 'Economy16']], 
                      happy_df_17[['country_txt','Happiness.Score17', 'Freedom17', 'Economy17']]]

###TRIALS

In [0]:
# d = {'one': pd.Series([1., 3.], index=['a', 'c']), 'two': pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd']), 'three': pd.Series(["A", "B", "A"], index=['a', 'c', 'd'])}
# df = pd.DataFrame(d)
# print(df)
# # print(df.isna().sum())
# df = df.fillna(df.mean())
# mode =  df['three'].mode()[0]
# df['three'] = df.groupby('three')['three'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
# df['three']= df['three'].fillna(df['three'].value_counts().idxmax())
# print(df)
# df = pd.DataFrame(data={'location': [1, 2, 3],
#                         'coor': [(14.48847, 103.161477),
#                               (14.970084, 103.062853),
#                               (np.nan, np.nan)]})
# print(df)
# # df.dropna(how='any')
# df = df[~df.coor.apply(lambda x: np.isnan(x[0]) & np.isnan(x[1]))]
# # df = df[pd.DataFrame(df.coor.tolist()).notna().all(1)]
# print(df)


## MERGED DATA

In [20]:
ethnic_count = geo_filtered_df.groupby('country_txt').count()
countries = pd.DataFrame(geo_filtered_df['country_txt'].drop_duplicates().sort_values())
ethnic_count = pd.concat([ethnic_count.reset_index(drop=True),countries.reset_index(drop=True)], axis=1)
ethnic_count = ethnic_count.rename(index=str, columns={"group":"group_count"})
print(ethnic_count.columns.values)


['group_count' 'country_txt']


In [21]:
#result = pd.concat([gtd_filtered_df, happy_filtered_df],gtd_filtered_df["country_txt"]=happy_filtered_df["Country"])
#'Happiness.Rank' 'Happiness.Score'
gtd_merged_happiness = gtd_filtered_df
for i in range(len(happiness_filtered)):
  gtd_merged_happiness = pd.merge(gtd_merged_happiness, happiness_filtered[i],on='country_txt')
gtd_merged_happiness = pd.merge(gtd_merged_happiness, ethnic_count, on='country_txt')
print(gtd_merged_happiness.shape)
# print(gtd_merged_happiness['region_txt'].drop_duplicates())

(167161, 21)


In [22]:
print(gtd_merged_happiness.columns.values)

['country_txt' 'region_txt' 'city' 'provstate' 'target1' 'iyear' 'imonth'
 'gname' 'weaptype1_txt' 'attacktype1_txt' 'attacktype1'
 'Happiness.Score15' 'Freedom15' 'Economy15' 'Happiness.Score16'
 'Freedom16' 'Economy16' 'Happiness.Score17' 'Freedom17' 'Economy17'
 'group_count']


In [0]:
# MENA stands for Middle East and North Africa region
mena = gtd_merged_happiness.loc[gtd_merged_happiness['region_txt'] == 'Middle East & North Africa']

In [24]:
print(mena.shape)

(45846, 21)


In [25]:
print(mena['country_txt'].drop_duplicates())

16236           Jordan
19871            Egypt
23165          Lebanon
25643           Turkey
30049             Iran
61678           Israel
64027           Kuwait
78871          Algeria
91750          Morocco
114426           Syria
119326            Iraq
153137    Saudi Arabia
154671         Bahrain
157616           Yemen
Name: country_txt, dtype: object


In [26]:
# Number of distinct terrorist groups in the MENA region = 636
# print(np.unique(mena[['gname']].values))
print(np.unique(mena[['gname']]).size)

636


In [0]:
# attribute "Unknown > 27000" I think we should drop all of that
# It doesn't make sense to predict an "Unknown"
# print(mena['gname'].value_counts())
# print(mena['gname'].size)

### Filtering MENA Dataframe

In [28]:
# drop rows with nan in mena
print(mena.shape)
mena_cleaned = mena.dropna()
print(mena_cleaned.shape)

# select desired labels
mena_filtered = mena_cleaned[(mena_cleaned['gname'] == "Unknown") | (mena_cleaned['gname'] == "Islamic State of Iraq and the Levant (ISIL)") | (mena_cleaned['gname'] == "Kurdistan Workers' Party (PKK)") | (mena_cleaned['gname'] == "Houthi extremists (Ansar Allah)") | (mena_cleaned['gname'] == "Al-Qaida in the Arabian Peninsula (AQAP)") | (mena_cleaned['gname'] == "Al-Qaida in Iraq")]

print(mena_filtered['gname'].value_counts())
print(mena_filtered['gname'].size)

labels = ["Unknown", "Islamic State of Iraq and the Levant (ISIL)", "Kurdistan Workers' Party (PKK)", "Houthi extremists (Ansar Allah)", "Al-Qaida in the Arabian Peninsula (AQAP)", "Al-Qaida in Iraq"]
labels1 = [1, 2, 3, 4, 5, 6]


(45846, 21)
(45624, 21)
Unknown                                        27132
Islamic State of Iraq and the Levant (ISIL)     5553
Kurdistan Workers' Party (PKK)                  2134
Houthi extremists (Ansar Allah)                 1062
Al-Qaida in the Arabian Peninsula (AQAP)        1016
Al-Qaida in Iraq                                 635
Name: gname, dtype: int64
37532


# Training

### Divide Data into Train & Test

In [29]:
# divide dataset into train and test
mena_train = mena_filtered.copy()
mena_test = mena_filtered.copy()

mena_train = mena_train[mena_train['iyear'] < 2017]
mena_test = mena_test[mena_test['iyear'] >= 2017]

print(mena_train.shape)
print(mena_test.shape)

(34309, 21)
(3223, 21)


###KNN Classifier

In [30]:
# KNN classifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors = 6)

#Train the model using the training sets
knn.fit(mena_train, labels1)

#Predict the response for test dataset
y_pred = knn.predict(mena_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(mena_test, y_pred))

ValueError: ignored

In [0]:
test = pd.DataFrame({'id': np.array([1,2,3,4] * 2,dtype='int32'),
                     'num': np.array([1,2] * 4,dtype='int32')
                    })
print(test)

test = test[(test['id'] != 1) & (test['num'] != 1)]
print(test)